In [20]:
%matplotlib inline
%matplotlib widget
import sympy as sym
import numpy as np

In [21]:
# Convert DH parameters to a homogeneous transformation matrix.
def dh_to_matrix(dh_param):
    a, alpha, d, theta = dh_param
    cos_theta = sym.cos(theta)
    sin_theta = sym.sin(theta)
    cos_alpha = sym.cos(alpha)
    sin_alpha = sym.sin(alpha)

    Ti = sym.Matrix([
        [cos_theta, -sin_theta, 0, a],
        [cos_alpha*sin_theta, cos_alpha*cos_theta, -sin_alpha, -d * sin_alpha],
        [sin_alpha*sin_theta, sin_alpha*cos_theta, cos_alpha, d*cos_alpha],
        [0, 0, 0, 1]
    ])
    return Ti

def forward_kinematics(dh_params):
    n = len(dh_params)
    T = sym.eye(4)
    T_list = [T]
    for i in range(n):
        Ti = dh_to_matrix(dh_params[i])
        T = T @ Ti
        T_list.append(T)
    return T_list

# Define the symbols
q1, q2, q3 = sym.symbols('q1 q2 q3')
l1, l2, l3 = sym.symbols('l1 l2 l3')

# Define the DH param
theta = np.array([q1, q2, q3])
L = np.array([l1,l2, l3])
dh_params = np.array([
    [0.0, 0.0, L[0], theta[0]],
    [0.0, theta[1], L[1], 0.0],
    [0.0, theta[2], L[2], 0.0]
    ])

In [22]:
p_current = forward_kinematics(dh_params)[-1]
p_current

Matrix([
[cos(q1),  sin(q1)*sin(q2)*sin(q3) - sin(q1)*cos(q2)*cos(q3),  sin(q1)*sin(q2)*cos(q3) + sin(q1)*sin(q3)*cos(q2),  l2*sin(q1)*sin(q2) + l3*sin(q1)*sin(q2)*cos(q3) + l3*sin(q1)*sin(q3)*cos(q2)],
[sin(q1), -sin(q2)*sin(q3)*cos(q1) + cos(q1)*cos(q2)*cos(q3), -sin(q2)*cos(q1)*cos(q3) - sin(q3)*cos(q1)*cos(q2), -l2*sin(q2)*cos(q1) - l3*sin(q2)*cos(q1)*cos(q3) - l3*sin(q3)*cos(q1)*cos(q2)],
[      0,                  sin(q2)*cos(q3) + sin(q3)*cos(q2),                 -sin(q2)*sin(q3) + cos(q2)*cos(q3),                     l1 + l2*cos(q2) - l3*sin(q2)*sin(q3) + l3*cos(q2)*cos(q3)],
[      0,                                                  0,                                                  0,                                                                             1]])

In [23]:
J = sym.eye(3)
J[:,0]= p_current.diff(q1)[:3,3]
J[:,1]= p_current.diff(q2)[:3,3]
J

Matrix([
[l2*sin(q2)*cos(q1) + l3*sin(q2)*cos(q1)*cos(q3) + l3*sin(q3)*cos(q1)*cos(q2),  l2*sin(q1)*cos(q2) - l3*sin(q1)*sin(q2)*sin(q3) + l3*sin(q1)*cos(q2)*cos(q3), 0],
[l2*sin(q1)*sin(q2) + l3*sin(q1)*sin(q2)*cos(q3) + l3*sin(q1)*sin(q3)*cos(q2), -l2*cos(q1)*cos(q2) + l3*sin(q2)*sin(q3)*cos(q1) - l3*cos(q1)*cos(q2)*cos(q3), 0],
[                                                                           0,                         -l2*sin(q2) - l3*sin(q2)*cos(q3) - l3*sin(q3)*cos(q2), 1]])

In [24]:
J.inv()

Matrix([
[                                                                       cos(q1)/(l2*sin(q1)**2*sin(q2) + l2*sin(q2)*cos(q1)**2 + l3*sin(q1)**2*sin(q2)*cos(q3) + l3*sin(q1)**2*sin(q3)*cos(q2) + l3*sin(q2)*cos(q1)**2*cos(q3) + l3*sin(q3)*cos(q1)**2*cos(q2)),                                                                         sin(q1)/(l2*sin(q1)**2*sin(q2) + l2*sin(q2)*cos(q1)**2 + l3*sin(q1)**2*sin(q2)*cos(q3) + l3*sin(q1)**2*sin(q3)*cos(q2) + l3*sin(q2)*cos(q1)**2*cos(q3) + l3*sin(q3)*cos(q1)**2*cos(q2)), 0],
[                                                                       sin(q1)/(l2*sin(q1)**2*cos(q2) + l2*cos(q1)**2*cos(q2) - l3*sin(q1)**2*sin(q2)*sin(q3) + l3*sin(q1)**2*cos(q2)*cos(q3) - l3*sin(q2)*sin(q3)*cos(q1)**2 + l3*cos(q1)**2*cos(q2)*cos(q3)),                                                                        -cos(q1)/(l2*sin(q1)**2*cos(q2) + l2*cos(q1)**2*cos(q2) - l3*sin(q1)**2*sin(q2)*sin(q3) + l3*sin(q1)**2*cos(q2)*cos(q3) - l3*sin(q2)*sin(q3)*cos(